In [4]:
"import necessary packages"
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import *

In [ ]:
"reimplement torch permute"
from functools import reduce,partial
product=partial(reduce,lambda x,y:x*y)

shape=[2,3,4,5]
mat=torch.arange(product(shape)).reshape(shape)
print("[origin]\n",mat)

#### torch implement ####
permute_1=mat.permute(0,1,3,2) #[omit] simply transpose the last two dimension

mat2=mat.permute(1,0,2,3)
# print("[permute(1,0,2,3)]\n",mat2)

mat3=mat.permute(2,1,0,3)
# print("[permute(2,1,0,3)]\n",mat3)

mat4=mat.permute(3,2,0,1)
# print("[permute(3,2,0,1)]\n",mat4)
#### torch implement ####

#### reimplement ####
def transpose_2d(tensor:torch.Tensor,new_dim_indices:Iterable=[1,0]):
    assert tensor.ndim == len(new_dim_indices), "length of dimension indices passed in must be equal to shape of tensor passed in."
    new_tensor=[]
    new_dims=[]
    for _ in new_dim_indices:
        new_dims.append(tensor.shape[_])

    for j in range(new_dims[0]):
        temp=[]
        for i in range(new_dims[1]):
            temp.append(tensor[i,j])
        new_tensor.append(temp)
    return torch.tensor(new_tensor,dtype=tensor.dtype,device=tensor.device)

temp_tensor=torch.arange(3*5).view(3,5).requires_grad_(False)
print(temp_tensor)
print(transpose_2d(temp_tensor,new_dim_indices=[1,0]))

def permute(tensor:torch.Tensor, dim_indices:Iterable):
    assert tensor.ndim == len(dim_indices), "length of dimension indices passed in must be equal to shape of tensor passed in."
    new_tensor=[]
    access_index=[None for _ in range(len(dim_indices))]
    def _recursive_permute(access_index):
        temp_tensor=[]
        for i in range(access_index):
            temp_tensor.append(tensor[i])
        return temp_tensor

#### reimplement ####


[origin]
 tensor([[[[  0,   1,   2,   3,   4],
          [  5,   6,   7,   8,   9],
          [ 10,  11,  12,  13,  14],
          [ 15,  16,  17,  18,  19]],

         [[ 20,  21,  22,  23,  24],
          [ 25,  26,  27,  28,  29],
          [ 30,  31,  32,  33,  34],
          [ 35,  36,  37,  38,  39]],

         [[ 40,  41,  42,  43,  44],
          [ 45,  46,  47,  48,  49],
          [ 50,  51,  52,  53,  54],
          [ 55,  56,  57,  58,  59]]],


        [[[ 60,  61,  62,  63,  64],
          [ 65,  66,  67,  68,  69],
          [ 70,  71,  72,  73,  74],
          [ 75,  76,  77,  78,  79]],

         [[ 80,  81,  82,  83,  84],
          [ 85,  86,  87,  88,  89],
          [ 90,  91,  92,  93,  94],
          [ 95,  96,  97,  98,  99]],

         [[100, 101, 102, 103, 104],
          [105, 106, 107, 108, 109],
          [110, 111, 112, 113, 114],
          [115, 116, 117, 118, 119]]]])
tensor([[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14

In [6]:
"reimplement CrossEntropyLoss"
reduction='mean'
num_predict=3
vocab_size=8

logits=torch.randint(0,10,(num_predict,vocab_size)).to(dtype=torch.float32)
label=torch.arange(num_predict).random_(vocab_size-1)
print("[logits]\n",logits)
print("[label]\n",label)

#### torch implement ####
loss_func=nn.CrossEntropyLoss(reduction=reduction) # alert that reduction of CrossEntropyLoss is mean
print("[loss]\n",loss_func(logits,label))
#### torch implement ####

#### reimplement ####
logits_dim_max=logits.amax(-1,keepdim=True)
after_diff_logits = logits-logits_dim_max # to avoid digits overflow 
### softmax ###
after_softmax_logits=after_diff_logits.exp()/after_diff_logits.exp().sum(-1,keepdim=True)
### softmax ###
### NLL: negative log-likehood ###
after_log_logits = -(after_softmax_logits.log())
### NLL: negative log-likehood ###
### calculate loss ###
before_reduction=after_log_logits[[i for i in range(num_predict)], label]
if reduction=='mean':
    loss=before_reduction.mean()
elif reduction=='sum':
    loss=before_reduction.sum()
elif reduction=='none':
    loss=before_reduction
### calculate loss ###
print("[loss(reimplement)] ",loss)
#### reimplement ####


[logits]
 tensor([[3., 1., 3., 7., 5., 1., 8., 2.],
        [8., 9., 9., 2., 5., 9., 7., 7.],
        [1., 0., 3., 6., 8., 1., 5., 5.]])
[label]
 tensor([6, 5, 2])
[loss]
 tensor(2.2922)
[loss(reimplement)]  tensor(2.2922)
